In [4]:
! pip install numpy scikit-learn tqdm

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 49.6 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 52.0 MB/s eta 0:00:00
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 54.0 MB/s eta 0:00:0000:0100:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [scikit-learn] [scikit-learn]


In [9]:
# ----------------------------------------------
# 1. Imports and helper utilities
# ----------------------------------------------
import numpy as np
from numpy.linalg import pinv, norm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from tqdm.auto import trange

def generate_data(n, p, theta, Sigma, rng):
    """
    Draw n labelled observations from the two-component anisotropic
    Gaussian mixture described in the paper.
    """
    labels = rng.choice([-1, 1], size=n)
    W = rng.multivariate_normal(np.zeros(p), Sigma, size=n)
    Y = labels[:, None] * theta + W
    return Y, labels

def bayes_predict(x, theta, Sigma_inv):
    """Bayes oracle for balanced classes (threshold at 0)."""
    return np.sign(x @ (Sigma_inv @ theta))

def lda_fit_predict(Y_train, y_train, Y_test):
    """
    Classical LDA using a pseudo-inverse when p > n.
    scikit-learn handles the pseudo-inverse internally.
    """
    clf = LinearDiscriminantAnalysis(store_covariance=True)
    clf.fit(Y_train, y_train)
    return clf.predict(Y_test)

def ols_min_norm(Y_train, y_train):
    """
    Interpolating minimum-norm solution:
        theta_hat = Y (YᵀY)⁺ y     (Moore–Penrose pseudo-inverse)
    """
    return Y_train.T @ pinv(Y_train @ Y_train.T) @ y_train

def evaluate(pred, true):
    return np.mean(pred != true)

# ----------------------------------------------
# 2. Core simulation loop
# ----------------------------------------------
def run_one_trial(n, p, delta, Sigma, rng):
    """
    One Monte-Carlo trial:
      • Draw theta with norm delta in the lowest-variance directions
      • Form training and test sets
      • Compare error of LDA, OLS interpolator, and the Bayes oracle
    """
    # Choose theta aligned with the last coordinate (lowest eigenvalue)
    theta = np.zeros(p)
    theta[-1] = delta

    # Pre-compute Sigma inverse once
    Sigma_inv = np.diag(1.0 / np.diag(Sigma))

    # Draw data
    Y_train, y_train = generate_data(n, p, theta, Sigma, rng)
    Y_test,  y_test  = generate_data(5000, p, theta, Sigma, rng)  # large test set

    # LDA
    y_pred_lda = lda_fit_predict(Y_train, y_train, Y_test)
    err_lda = evaluate(y_pred_lda, y_test)

    # Interpolating OLS
    theta_ols = ols_min_norm(Y_train, y_train)
    err_ols = evaluate(np.sign(Y_test @ theta_ols), y_test)

    # Bayes oracle (needs true parameters)
    y_pred_bayes = bayes_predict(Y_test, theta, Sigma_inv)
    err_bayes = evaluate(y_pred_bayes, y_test)

    return err_lda, err_ols, err_bayes

def simulate(
    n=100,
    p_list=(100, 200, 400, 800, 1600),
    delta=3.0,
    sigma_large=3.0,
    sigma_small=1.0,
    trials=10,
    seed=0
):
    """
    Repeat the experiment over multiple ambient dimensions.
    Sigma is diagonal with a pronounced anisotropy: first half
    coordinates have large variance, remainder have small variance.
    """
    rng = np.random.default_rng(seed)
    results = {}

    for p in p_list:
        # Build anisotropic diagonal covariance
        diag = np.concatenate([np.full(p // 2, sigma_large ** 2),
                               np.full(p - p // 2, sigma_small ** 2)])
        Sigma = np.diag(diag)

        errs_lda, errs_ols, errs_bayes = [], [], []
        for _ in trange(trials, leave=False):
            e1, e2, e3 = run_one_trial(n, p, delta, Sigma, rng)
            errs_lda.append(e1)
            errs_ols.append(e2)
            errs_bayes.append(e3)

        results[p] = {
            "lda":   np.mean(errs_lda),
            "ols":   np.mean(errs_ols),
            "bayes": np.mean(errs_bayes)
        }
    return results

# ----------------------------------------------
# 3. Example call (prints a table)
# ----------------------------------------------
if __name__ == "__main__":
    res = simulate()
    print("Average mis-classification rate over 10 trials\n")
    print(" p\tLDA\tOLS-min-norm\tBayes")
    for p, val in res.items():
        print(f"{p}\t{val['lda']:.3f}\t{val['ols']:.3f}\t{val['bayes']:.3f}")

Average mis-classification rate over 10 trials

 p	LDA	OLS-min-norm	Bayes
100	0.261	0.439	0.001
200	0.091	0.102	0.001
400	0.147	0.195	0.001
800	0.296	0.300	0.001
1600	0.387	0.354	0.001
